Маючи дані з sellingPartnerSearchTermsReport  потрібно побудувати таблицю таким чином щоб порядок зберігся оскільки дані вже відсортовані згідно searchFrequencyRank. Також при побудові таблиці потрібно буде врахувати що відношення кількості замовлень до кількості переглядів не може бути більше за 1 (тобто 100%) а в середньому воно коливається від 1 до 25 відсотків.. Згідно цих даних конверсія - це кількість замовлень.. Нам потрібно підібрати загальну кількість замовлень загальну кількість переглядів та відношення...  Будемо працювати з бібліотекою pandas яка є дуже гнучкою для роботи з таблицями

Побудуємо функцію для отримання даних з файлу та виконаємо її

In [210]:
import os
import pandas as pd
import numpy as np

def process_excel_data(file_path):
    """
    Зчитує Excel-файл, перейменовує стовпці та обчислює конверсії.

    Параметри:
        file_path (str): Шлях до Excel-файлу.

    Повертає:
        pd.DataFrame: Оброблений DataFrame.

    Викидає:
        FileNotFoundError: Якщо файл не знайдено.
        ValueError: Якщо виникає помилка при зчитуванні файлу.
    """
    # Перевірка наявності файлу
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {file_path} не знайдено.")
    
    try:
        # Зчитуємо дані з Excel-файлу
        df = pd.read_excel(file_path, skiprows=1, header=0)
    except Exception as e:
        raise ValueError(f"Помилка при зчитуванні файлу: {e}")
    
    # Перейменовуємо стовпці
    new_columns = [
        'Department', 'Search_Term', 'Search_Frequency_Rank', 'ASIN_1',
        'Product_Title_1', 'Click_Share_1', 'Conversion_Share_1',
        'ASIN_2', 'Product_Title_2', 'Click_Share_2', 'Conversion_Share_2',
        'ASIN_3', 'Product_Title_3', 'Click_Share_3', 'Conversion_Share_3'
    ]
    df.columns = new_columns
    
    # Обчислюємо конверсії (множимо значення на 100)
    df['Conversion_Share_1'] = df['Conversion_Share_1'] * 100
    df['Conversion_Share_2'] = df['Conversion_Share_2'] * 100
    df['Conversion_Share_3'] = df['Conversion_Share_3'] * 100
    df['Click_Share_1'] = df['Click_Share_1'] * 100
    df['Click_Share_2'] = df['Click_Share_2'] * 100
    df['Click_Share_3'] = df['Click_Share_3'] * 100
    
    return df

In [301]:
file_path1 = r'C:\Users\user\Downloads\brandAnalytics\data7days\15.xls'
df = process_excel_data(file_path1)
df.head()

,Department,Search_Term,Search_Frequency_Rank,ASIN_1,Product_Title_1,Click_Share_1,Conversion_Share_1,ASIN_2,Product_Title_2,Click_Share_2,Conversion_Share_2,ASIN_3,Product_Title_3,Click_Share_3,Conversion_Share_3
0,Amazon.com,christmas decorations,1,B08B5PHMH6,Ivenf Christmas Decorations Outdoor Yard Front...,4.4935,4.3560,B07HHBQJGJ,LUDILO 135Pcs Christmas Window Clings Snowflak...,3.6423,8.3372,B08CVSGC2T,HAJACK Christmas Pillow Covers & Christmas Dec...,2.9974,3.9813
1,Amazon.com,christmas tree,2,B08J7JRJL3,Best Choice Products 6ft Pre-Lit Pre-Decorated...,6.8103,9.2506,B0BHTTZ9H9,Best Choice Products 4.5ft Pre-Lit Snow Flocke...,5.1794,4.6253,B08D9BGRWP,Goplus 5ft Artificial Christmas Tree Xmas Pine...,4.9201,4.7424
2,Amazon.com,black friday deals 2022,3,B0BKGQR2SV,Leggings For Women Fleece Lined Winter Warm Th...,5.4312,8.6957,B09F39HQ46,Christmas Pajamas for Family Red Plaid Elk Xma...,4.4176,3.8261,B09SB1HHZ4,PHIXTON Rechargeable High Lumens Tactical Hand...,3.1814,5.0435
3,Amazon.com,sweaters for women,4,B08FLSJHZD,Ugerlov Women's Oversized Sweaters Batwing Sle...,2.0320,0.9459,B0828RW3HM,ZAFUL Women's Cropped Turtleneck Sweater Lante...,1.2034,0.0000,B098MPX79Z,Ecrocoo Rose Sweater for Women Fashion Casual ...,0.8187,0.5405
4,Amazon.com,squishmallow,5,B08CGCH5Q2,Squishmallows Official Kellytoy Plush 8 Plush ...,8.9563,11.5921,B09NB1W7X3,Squishmallows 12-Inch Purple and Pink Spotted ...,3.5557,7.0707,B09NB2BFNH,Squishmallows 12-Inch Pancake Plush - Add Raye...,2.7107,2.5493


In [302]:
new_df = df.copy()

Алгоритм такий: починаємо роботу з останнього рядка так як в даному search_term кількість запитів найменша.(Працюємо тільки з тими рядками в яких хоч один з асінів має замовлення) В циклі підбираємо кількості замовлень по кожному асіну та потім загальну кількість замовлень та загальну кількість запитів search_volume та присвоюємо це значення змінній prev_search_volume Далі працюємо з наступним рядком підбираємо кількості замовлень і тд і порівнюємо search_volume з попереднім Якщо умовИ (1-наступний search_volume відрізняється від попереднього не більше чим на 8%, 2-відношення кількості замовлень до кількості переглядів не більше за 24% .Згідно статистики відношення кількості замовлень до кількості переглядів коливаються в межах до 25%) виконуються то ми ці дані записуємо в результуючий датафрейм переприсвоюємо search_volume та виходимо з циклу. Далі переходимо до наступного Search_Term по зростанню .Якщо всі conversion_share по трьом асінам рівні 0 ми пропускаємо даний рядок. В кінці ми сортуємо датафрейм по спаданню

In [335]:
def process_row(row, min_conversion_share, prev_search_volume):
    should_break=True
    i=1
    while should_break:
        nsd = min_conversion_share / i
        a1 = round(row['Conversion_Share_1'] / nsd)
        a2 = round(row['Conversion_Share_2'] / nsd)
        a3 = round(row['Conversion_Share_3'] / nsd)
        others = round((100 - (row['Conversion_Share_1'] + row['Conversion_Share_2'] + row['Conversion_Share_3'])) / nsd)
        values = [a1, a2, a3, others]
        total_orders = sum(values)
        values.append(total_orders)

        i+=0.2
        
        # Обчислюємо searchVolume та conversion_share
        for j in range(7,50):
            conversion_share_others = 100 - (row['Conversion_Share_1'] + row['Conversion_Share_2'] + row['Conversion_Share_3'])
            click_volume = values[3]*j
            search_volume = round((others / conversion_share_others) * 100)
            
            # Перевіряємо, чи search_volume1 більший за попередній searchVolume
            if search_volume >= prev_search_volume :
                # Обчислюємо conversion
                orders = values[-1]
                conversion_share = round(total_orders/ search_volume, 6) if total_orders != 0 else 0
                c1 = a1/(search_volume*row['Click_Share_1']*0.01)
                c2 = a2/(search_volume*row['Click_Share_2']*0.01)
                c3 = a3/(search_volume*row['Click_Share_3']*0.01)
                
                if conversion_share < 0.24  and search_volume < prev_search_volume*1.06 and all(c < 0.5 for c in [c1, c2, c3]):
                    values.extend([c1,c2,c3])
                    return search_volume, conversion_share, values
                if total_orders > prev_search_volume:
                    return search_volume, conversion_share, values
                if search_volume > prev_search_volume*1.06:
                    break
        if i>100 or search_volume<=total_orders*0.3:
            return search_volume, total_orders, values
    

             

def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Обробляє DataFrame, починаючи з останнього рядка, і обчислює searchVolume та conversion.
    Зупиняється, коли searchVolume поточного рядка стає більшим за попередній.
    """
    # Сортуємо DataFrame за Search_Frequency_Rank за зростанням (останній рядок перший)
    df = df.sort_values('Search_Frequency_Rank', ascending=False).reset_index(drop=True)
    stop_flag = False
    for i in range(df.shape[0]):
        if stop_flag:
            break
        first_row = df.iloc[i]
        conversions = list(filter(None, [df.at[i,'Conversion_Share_1'], df.at[i,'Conversion_Share_2'], df.at[i,'Conversion_Share_3']]))
        # Перевіряємо, чи список порожній
        if not conversions:
            continue  # Переходимо до наступної ітерації
        # Обчислюємо min_conversion
       # k = i
        nsd = min(conversions)
        a1 = round(df.at[i,'Conversion_Share_1'] / nsd)
        a2 = round(df.at[i,'Conversion_Share_2'] / nsd)
        a3 = round(df.at[i,'Conversion_Share_3'] / nsd)
        others = round((100 - (df.at[i,'Conversion_Share_1'] + df.at[i,'Conversion_Share_2'] + df.at[i,'Conversion_Share_3'])) / nsd)
        total_orders = a1 + a2 + a3 + others
        for j in range(6,20):       
            max_conversion = max(df.at[i,'Conversion_Share_1'], df.at[i,'Conversion_Share_2'], df.at[i,'Conversion_Share_3'])
            # Визначаємо, який стовпець містить максимальне значення
            if max_conversion == df.at[i,'Conversion_Share_1']:
                click_volume = a1*j
                search_volume = round((click_volume / df.at[i,'Click_Share_1']) * 100)
            elif max_conversion == df.at[i,'Conversion_Share_2']:
                click_volume = a2*j
                search_volume = round((click_volume / df.at[i,'Click_Share_2']) * 100) 
            elif max_conversion == df.at[i,'Conversion_Share_3']:
                click_volume = a3*j
                search_volume = round((click_volume / df.at[i,'Click_Share_3']) * 100)
            # Перевіряємо, чи search_volume1 більший за попередній searchVolume
            conversion_share = round(total_orders/ search_volume, 6)
            word_count = len(df.iloc[i,1].split()) 
            if (word_count > 1 and conversion_share < 0.22) or (word_count == 1 and conversion_share < 0.02):
                c1 = a1/(search_volume*df.at[i,'Click_Share_1']*0.01)
                c2 = a2/(search_volume*df.at[i,'Click_Share_2']*0.01)
                c3 = a3/(search_volume*df.at[i,'Click_Share_3']*0.01)
                result_df = pd.DataFrame(columns=['Search_Term', 'totalOrders', 'conversion_share', 'searchVolume', 'conversion_asin1', 'conversion_asin2', 'conversion_asin3'])
                result_df = result_df.astype({'Search_Term': 'object', 'totalOrders': 'int64', 'conversion_share': 'float64', 'searchVolume': 'int64', 'conversion_asin1': 'float64', 'conversion_asin2': 'float64', 'conversion_asin3': 'float64'})
                # Дані для нового рядка
                new_row = {
                    'Search_Term': df.iloc[i, 1],
                    'totalOrders': total_orders,
                    'conversion_share': conversion_share,
                    'searchVolume': search_volume,
                    'conversion_asin1': c1,
                    'conversion_asin2': c2,
                    'conversion_asin3': c3
                }             
                # Додаємо перший рядок
                result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
                stop_flag = True
                break
                
    df = df.iloc[i+1:].reset_index(drop=True)
    # Ініціалізуємо попередній searchVolume 
    prev_search_volume = result_df.at[0, 'searchVolume']  # Безпечно, бо індекси починаються з 0
    print(prev_search_volume)
    
    for _, row in df.iterrows():
        conversions = list(filter(None, [row['Conversion_Share_1'], row['Conversion_Share_2'], row['Conversion_Share_3']]))
        # Перевіряємо, чи список порожній
        if not conversions:
            continue  # Переходимо до наступної ітерації
        # Обчислюємо min_conversion
        min_conversion_share = min(conversions)
        should_break = True
        i=1
        # Обчислюємо можливі замовлення
        
        
        while should_break:
            nsd = min_conversion_share / i
            a1 = round(row['Conversion_Share_1'] / nsd)
            a2 = round(row['Conversion_Share_2'] / nsd)
            a3 = round(row['Conversion_Share_3'] / nsd)
            others = round((100 - (row['Conversion_Share_1'] + row['Conversion_Share_2'] + row['Conversion_Share_3'])) / nsd)
            values = [a1, a2, a3, others]
            total_orders = sum(values)
            values.append(total_orders)
            #if total_orders > prev_search_volume/2:
            #    break
            i+=0.1
            # Обчислюємо searchVolume та conversion_share
            for j in range(6,16):       
                max_conversion = max(row['Conversion_Share_1'], row['Conversion_Share_2'], row['Conversion_Share_3'])
                # Визначаємо, який стовпець містить максимальне значення
                if max_conversion == row['Conversion_Share_1']:
                    click_volume = values[0]*j
                    search_volume = round((click_volume / row['Click_Share_1']) * 100)
                elif max_conversion == row['Conversion_Share_2']:
                    click_volume = values[1]*j
                    search_volume = round((click_volume / row['Click_Share_2']) * 100) 
                elif max_conversion == row['Conversion_Share_3']:
                    click_volume = values[2]*j
                    search_volume = round((click_volume / row['Click_Share_3']) * 100)
                # Перевіряємо, чи search_volume1 більший за попередній searchVolume
                if search_volume >= prev_search_volume :
                    # Обчислюємо conversion    
                    orders = values[4]
                    conversion_share = round(orders/ search_volume, 6) if orders != 0 else 0
                    c1 = a1/(search_volume*row['Click_Share_1']*0.01)
                    c2 = a2/(search_volume*row['Click_Share_2']*0.01)
                    c3 = a3/(search_volume*row['Click_Share_3']*0.01)
                    if search_volume >= prev_search_volume*1.11:
                        break
                    if conversion_share < 0.24  and search_volume <= prev_search_volume*1.08 and all(c < 0.5 for c in [c1, c2, c3]):
                        values.extend([c1,c2,c3])
                        should_break = False
                        break
                    if search_volume <  (total_orders*3):
                        print(search_volume, row['Search_Term'])
                        print(values[4])
                        should_break = False
                        break    
            if i>=200 :
               should_break = False 
                        
        if conversion_share > 0.24  or search_volume > prev_search_volume*1.08 or any(c > 0.5 for c in [c1, c2, c3]):
            search_volume, conversion_share, values = process_row(row, min_conversion_share, prev_search_volume)
            if len(values)<7:
                continue
                
        word_count = len(row['Search_Term'].split()) 
        if word_count > 1 or (word_count == 1 and conversion_share < 0.02):
            # Додаємо результат до result_df
            c_s1 = values[-3]
            c_s2 = values[-2]
            c_s3 = values[-1]
            
            new_row = pd.DataFrame([[row['Search_Term'], orders, conversion_share, search_volume, c_s1, c_s2, c_s3]], 
                                  columns=['Search_Term', 'totalOrders', 'conversion_share', 'searchVolume', 'conversion_asin1', 'conversion_asin2', 'conversion_asin3'])
            result_df = pd.concat([result_df, new_row], ignore_index=True)
            # Оновлюємо попередній searchVolume
            prev_search_volume = search_volume

               
                               
    # Сортуємо результат за searchVolume за спаданням та скидаємо індекс
    result_df = result_df.sort_values('searchVolume', ascending=False).reset_index(drop=True)
    return result_df

In [336]:
process_df = process_data(new_df)
process_df.head(10)

282
(236, 7)


,Search_Term,totalOrders,conversion_share,searchVolume,conversion_asin1,conversion_asin2,conversion_asin3
0,christmas decorations,716,0.028976,24710,0.027919,0.066666,0.039154
1,christmas tree,1207,0.048928,24669,0.066665,0.043829,0.046962
2,black friday deals 2022,1019,0.041456,24580,0.066667,0.035917,0.065218
3,sweaters for women,3442,0.141297,24360,0.066667,0.000000,0.095269
4,my orders,563,0.023169,24300,0.022438,0.066668,0.026874
5,yellowstone season 5,1225,0.050418,24297,0.066666,0.046075,0.053095
6,sweater dress for women,1342,0.055279,24277,0.000000,0.033945,0.066668
7,womens sweaters,2700,0.112613,23976,0.066667,0.079959,0.101499
8,boots for women,1462,0.061008,23964,0.066665,0.018605,0.046053
9,iphone 14 pro max case,1189,0.050002,23779,0.066666,0.059330,0.065531


In [337]:
process_df['conversion_share'].mean()


np.float64(0.06216118220338983)

In [338]:
process_df.tail(10)

,Search_Term,totalOrders,conversion_share,searchVolume,conversion_asin1,conversion_asin2,conversion_asin3
226,floating shelves,17,0.043147,394,0.022810,0.033085,0.083252
227,portable charger,33,0.086162,383,0.090936,0.097471,0.043982
228,earrings for women,19,0.052486,362,0.056596,0.050690,0.111086
229,ear buds wireless bluetooth earbuds,35,0.101744,344,0.142901,0.068010,0.064001
230,iphone 13,32,0.093567,342,0.124999,0.023352,0.356626
231,blanket hoodie,34,0.105919,321,0.166710,0.115816,0.063924
232,yoga mat,17,0.056291,302,0.071374,0.057321,0.059962
233,socks for women,23,0.076412,301,0.100056,0.061389,0.074438
234,computer desk,20,0.066890,299,0.083358,0.049026,0.058697
235,jewelry box,29,0.102837,282,0.166797,0.061119,0.090842


In [339]:
# Знаходимо рядки, де хоча б одне зі значень у колонках conversion_asin1, conversion_asin2, conversion_asin3 більше за 0.5
outliers_df = process_df[
    (process_df['conversion_asin1'] > 0.3) |
    (process_df['conversion_asin2'] > 0.3) |
    (process_df['conversion_asin3'] > 0.3) |
    (process_df['conversion_share'] > 0.3)
]

# Виводимо результат
outliers_df

,Search_Term,totalOrders,conversion_share,searchVolume,conversion_asin1,conversion_asin2,conversion_asin3
230,iphone 13,32,0.093567,342,0.124999,0.023352,0.356626


In [342]:
#видаляємо аутлаєри
filtered_df = process_df[
    (process_df['conversion_asin1'] <= 0.3) &
    (process_df['conversion_asin2'] <= 0.3) &
    (process_df['conversion_asin3'] <= 0.3) &
    (process_df['conversion_share'] <= 0.3)
]
filtered_df.head()

,Search_Term,totalOrders,conversion_share,searchVolume,conversion_asin1,conversion_asin2,conversion_asin3
0,christmas decorations,716,0.028976,24710,0.027919,0.066666,0.039154
1,christmas tree,1207,0.048928,24669,0.066665,0.043829,0.046962
2,black friday deals 2022,1019,0.041456,24580,0.066667,0.035917,0.065218
3,sweaters for women,3442,0.141297,24360,0.066667,0.000000,0.095269
4,my orders,563,0.023169,24300,0.022438,0.066668,0.026874


In [343]:
res_df = filtered_df[['Search_Term', 'totalOrders', 'conversion_share', 'searchVolume']]
res_df.head(10)

,Search_Term,totalOrders,conversion_share,searchVolume
0,christmas decorations,716,0.028976,24710
1,christmas tree,1207,0.048928,24669
2,black friday deals 2022,1019,0.041456,24580
3,sweaters for women,3442,0.141297,24360
4,my orders,563,0.023169,24300
5,yellowstone season 5,1225,0.050418,24297
6,sweater dress for women,1342,0.055279,24277
7,womens sweaters,2700,0.112613,23976
8,boots for women,1462,0.061008,23964
9,iphone 14 pro max case,1189,0.050002,23779


Над цим алгоритмом потрібно ще працювати та вдосконалювати. Буду радий діалогу по даному питанню.